To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `hermes-function-calling-thinking-V1` from [Jofthomas](https://huggingface.co/datasets/Jofthomas/hermes-function-calling-thinking-V1), which is a toy version of 3K of the original [hermes-function-calling dataset](https://huggingface.co/datasets/NousResearch/hermes-function-calling-v1). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
from unsloth.chat_templates import get_chat_template

custom_template = \
    "{{bos_token}}"\
    "{% if messages[0]['role'] == 'system' %}"\
        "{{'>>> User: ' + messages[0]['content'] + '\nAlso, before making a tool call, take your time to plan in the format <think>your thoughts</think>\n\n' + messages[1]['content']}}"\
        "{% set loop_messages = messages[2:] %}"\
    "{% else %}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'human' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'model' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% elif message['role'] == 'tool' %}"\
            "{{ '>>> Tool: ' + message['content'] + '\n' }}"\
        "{% else %}"\
            "{{ raise_exception('Only user, assistant and tools roles are supported!') }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: <think>' }}"\
    "{% endif %}"

unsloth_eos_token = "eos_token"

tokenizer = get_chat_template(
    tokenizer,
    chat_template = (custom_template, unsloth_eos_token,), # You must provide a template and EOS token
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Jofthomas/hermes-function-calling-thinking-V1", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,).select_columns(['text'])

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Tokenizing train dataset (num_proc=2):   0%|          | 0/3570 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/3570 [00:00<?, ? examples/s]

In [6]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.748 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,570 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.271100
2,1.265200
3,1.358600
4,1.325500
5,1.222100
6,1.078900
7,1.043400
8,0.907800
9,0.794100
10,0.804100


In [8]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1436.8319 seconds used for training.
23.95 minutes used for training.
Peak reserved memory = 7.615 GB.
Peak reserved memory for training = 1.867 GB.
Peak reserved memory % of max memory = 51.659 %.
Peak reserved memory for training % of max memory = 12.665 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the query and tools avaliable.

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

<a name="Tool Calling"></a>
#### Tool Calling Utils

In [9]:
import re
from typing import List, Dict, Any, Callable
import inspect

def extract_tag_content(text, tag):
    matches = re.findall(fr"<{tag}>(.*?)</{tag}>", text, re.DOTALL)
    if not matches:
        raise Exception(f"Tag '{tag}' not found in the text.")
    return matches[-1]

def function_to_tool(func: Callable) -> Dict[str, Any]:
    signature = inspect.signature(func)
    properties = {}
    required = []

    for name, param in signature.parameters.items():
        param_type = param.annotation.__name__.lower() if param.annotation != param.empty else "string"
        properties[name] = {"type": param_type, "description": f"The {name} of the {func.__name__.replace('_', ' ')}"}
        required.append(name)

    desc = func.__doc__ or f'Get {func.__name__.replace("_", " ")}'

    return {
        "name": func.__name__,
        "description": f"{desc}",
        "parameters": {
            "type": "object",
            "properties": properties,
            "required": required
        }
    }

def get_tool_data(functions) -> List[Dict[str, Any]]:
    return [{"type": "function", "function": function_to_tool(func)} for func in functions]

def get_tool_result(tool_call: Dict[str, Any], available_functions: List[Callable]) -> Any:
    avaliable_lookup = {func.__name__: func for func in available_functions}
    tool_name = tool_call.get('name')
    tool_args = tool_call.get('arguments', {})
    if tool_name not in avaliable_lookup.keys():
        raise ValueError(f"Function '{tool_name}' not found.")

    func = avaliable_lookup[tool_name]
    try:
        return func(**tool_args)
    except Exception as e:
        return f"Error executing '{tool_name}': {str(e)}"


<a name="FakeToolDefinition"></a>
#### Fake tools
Some fake tools for testing purposes

In [28]:
import random

def create_user_profile(name, age, email):
    """Create a user profile with the given name, age, and email."""
    return {'status': 'success', 'message': f"User profile {name} successfully created"}

def generate_random_number(min_value, max_value):
    """Generate a random number between min_value and max_value."""
    return {'result': random.randint(int(min_value), int(max_value))}

def get_random_joke():
    """Get a random joke."""
    return {"Setup": "How do robots eat pizza?", "Punchline": "One byte at a time!"}

def get_stock_price(company: str):
    """Get the current stock price of a company"""
    if company == "Nvidia":
        return {'price': '200', 'currency': 'dollars'}
    elif company == "Apple":
        return {'price': '180', 'currency': 'dollars'}
    else:
        return None

avaliable_functions = [ generate_random_number, get_random_joke, get_stock_price ]
tools_description = get_tool_data(avaliable_functions)

queries = ["Tell me a joke", "Whats the stock price of Apple", "Give me a random number between 1002 and 2045"]

<a name="RunInference"></a>
#### Run Inference

In [21]:
user_prefix = """\
You are a function calling AI model. \
You are provided with function signatures within <tools></tools> XML tags. \
You may call one or more functions to assist with the user query. \
Don't make assumptions about what values to plug into functions. \
Here are the available tools: \
<tools>"""

tool_suffix = """\
</tools> \
Use the following pydantic model json schema for each tool call you will make: \
{\
'title': 'FunctionCall', \
'type': 'object', \
'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, \
'name': {'title': 'Name', 'type': 'string'}}, \
'required': ['arguments', 'name']\
} \
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call> \
Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>
"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def tool_query(query, n_depth:int=4):
  print(f"{'='*15}{query}{'='*15}")
  history = [{"role": "human", "content": f"{user_prefix}{str(tools_description)}{tool_suffix}\n\n{query}"}]
  for func_depth in range(n_depth):
    prompt = tokenizer.apply_chat_template(history, add_generation_prompt=True, tokenize=False)
    if func_depth > 0:
      prompt = prompt.removesuffix('<think>')
    inputs = tokenizer(prompt, return_tensors = "pt", add_special_tokens=False).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
    latest_step = tokenizer.batch_decode(outputs)[0].split(">>> Assistant: ")[-1].strip()
    history.append({"role":"model","content":latest_step})
    print("model generation:", latest_step)
    if "<tool_call>" not in latest_step:
      break
    try:
      tool_request = eval(extract_tag_content(latest_step, 'tool_call'))
      tool_result = get_tool_result(tool_request, avaliable_functions)
      history.append({"role":"tool","content": str(tool_result)})
      print("tool called by llm:", tool_request)
      print("tool result by python:", tool_result)
    except Exception as e:
      print(e)
      break
  return history

In [33]:
answer = tool_query("Tell me a funny quip")

===============Tell me a funny quip===============
model generation: <think>Okay, so the user asked for a funny quip. I need to figure out how to respond. Looking at the tools available, there's a function called get_random_joke which seems perfect for this. It doesn't require any parameters, so I can just call it without any arguments. That should provide the user with a random joke to make them laugh.
</think><tool_call>
{'name': 'get_random_joke', 'arguments': {}}
</tool_call><|end_of_text|>
tool called by llm: {'name': 'get_random_joke', 'arguments': {}}
tool result by python: {'Setup': 'How do robots eat pizza?', 'Punchline': 'One byte at a time!'}
model generation: One byte at a time!<|end_of_text|>


In [30]:
answer = tool_query("Whats get the price of Nvidia stock please")

===============Whats get the price of Nvidia stock please===============
model generation: <think>Okay, so the user is asking for the price of Nvidia stock. Let me see. I have the get_stock_price function available which takes a company name as an argument. The user mentioned Nvidia, so I should use that as the company parameter. The function call should look like this: {'name': 'get_stock_price', 'arguments': {'company': 'Nvidia'}}. That should get the current stock price for Nvidia.
</think><tool_call>
{'name': 'get_stock_price', 'arguments': {'company': 'Nvidia'}}
</tool_call><|end_of_text|>
tool called by llm: {'name': 'get_stock_price', 'arguments': {'company': 'Nvidia'}}
tool result by python: {'price': '200', 'currency': 'dollars'}
model generation: The current stock price of Nvidia is $200.<|end_of_text|>


In [32]:
answer = tool_query("Give me a random number between 1002 and 2045")

===============Give me a random number between 1002 and 2045===============
model generation: <think>Okay, I need to figure out how to respond to the user's request for a random number between 1002 and 2045. Looking at the available functions, there's a function called generate_random_number that fits perfectly here. It requires min_value and max_value, both of which the user provided. I'll call this function with the specified arguments to generate the number the user wants.
</think><tool_call>
{'name': 'generate_random_number', 'arguments': {'min_value': '1002','max_value': '2045'}}
</tool_call><|end_of_text|>
tool called by llm: {'name': 'generate_random_number', 'arguments': {'min_value': '1002', 'max_value': '2045'}}
tool result by python: {'result': 1273}
model generation: 1273<|end_of_text|>


 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [{"role": "human", "content": user_prefix + str(tools_description) + tool_suffix + '\n\n' + "What is Johnson&Johnsons's stock price as of right now?"}]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(prompt, return_tensors = "pt", add_special_tokens=False).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 300)

<|begin_of_text|>>>> User: You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'generate_random_number', 'description': 'Generate a random number between min_value and max_value.', 'parameters': {'type': 'object', 'properties': {'min_value': {'type':'string', 'description': 'The min_value of the generate random number'},'max_value': {'type':'string', 'description': 'The max_value of the generate random number'}},'required': ['min_value','max_value']}}}, {'type': 'function', 'function': {'name': 'get_random_joke', 'description': 'Get a random joke.', 'parameters': {'type': 'object', 'properties': {},'required': []}}}, {'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price o

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [16]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [17]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [{"role": "human", "content": user_prefix + str(tools_description) + tool_suffix + '\n\n' + "Can you generate a random number for me larger than 100"}]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(prompt, return_tensors = "pt", add_special_tokens=False).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 300)

<|begin_of_text|>>>> User: You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'generate_random_number', 'description': 'Generate a random number between min_value and max_value.', 'parameters': {'type': 'object', 'properties': {'min_value': {'type':'string', 'description': 'The min_value of the generate random number'},'max_value': {'type':'string', 'description': 'The max_value of the generate random number'}},'required': ['min_value','max_value']}}}, {'type': 'function', 'function': {'name': 'get_random_joke', 'description': 'Get a random joke.', 'parameters': {'type': 'object', 'properties': {},'required': []}}}, {'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price o

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [18]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [19]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [20]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
